# ANN via Annoy

In this notebook, we will use a public Japanese company name & address dataset to try the ANN search via Annoy, a lib made by Spotify.

Before importing below library, you may need to install fasttext by following the step [here](https://fasttext.cc/docs/en/support.html#building-fasttext-python-module).

In [1]:
import annoy
import pandas as pd
import numpy as np
import fasttext.util

Setup some variables

In [14]:
# we will demonstrate with 1000 samples first.
# Change the name to others if you are using the full dataset
npz_name = 'sample_1000.npz'

## Load the dataset

> NOTE: skip to "Load the cache file" if you already saved a npz cache

Here we use a public dataset containing all company names and addresses in Japan.  You can download the dataset from [here](https://info.gbiz.go.jp/hojin/DownloadTop).

In [46]:
data = pd.read_csv("./tmp/converted_company_ds.csv", dtype=str)


In [47]:
# convert NaN to empty string
data = data.fillna("")
data.sample(5)

,法人名,郵便番号,本社所在地,concat_name
69309,合同会社アミスター,086-1137,北海道標津郡中標津町字俵橋1425番地1,合同会社アミスター 086-1137 北海道標津郡中標津町字俵橋1425番地1
1083975,株式会社ザ・グリーン,170-0011,東京都豊島区池袋本町3丁目19番5池袋マンションヒルハイム302号,株式会社ザ・グリーン 170-0011 東京都豊島区池袋本町3丁目19番5池袋マンションヒル...
3380912,株式会社元気,865-0057,熊本県玉名市小浜577番地,株式会社元気 865-0057 熊本県玉名市小浜577番地
1910772,株式会社YOU&I,921-8044,石川県金沢市米泉町8丁目147番地3,株式会社YOU&I 921-8044 石川県金沢市米泉町8丁目147番地3
1963744,有限会社マルエツ,400-0117,山梨県甲斐市西八幡2295番地7,有限会社マルエツ 400-0117 山梨県甲斐市西八幡2295番地7


We will merge use the `concat_name` for converting into embedding.

In [48]:
# Here is a subset of converted data
# NOTE: remove the array slicing if you want to try the whole dataset
# features = data["concat_name"].sample(1000).values
features = data["concat_name"].values
print(features.shape)

(5167760,)


In [49]:
# Delete the df to save memory...
del(data)

-----
## Convert to Embedding

Because AnnoyIndex expects vectors as input.
We need to convert the text dataset into embedding first.
Here we use word vectors pre-trained by fastText [here](https://fasttext.cc/docs/en/crawl-vectors.html).

In [50]:
# Download the japanese gz
fasttext.util.download_model('ja', if_exists='ignore')

'cc.ja.300.bin'

In [51]:
# Load the pre-trained model
ft = fasttext.load_model('cc.ja.300.bin')

Get a feeling of how it looks like after conversion.

In [52]:
# A sample of the word vector inside
print("original text: ", features[0])
print(ft.get_word_vector(features[0])[:10])

original text:  釧路検察審査会 085-0824 北海道釧路市柏木町4-7
[ 0.00846744 -0.00213045  0.01018206 -0.017642    0.00224876 -0.00461199
 -0.00132453  0.00140697  0.0024185  -0.00132811]


It even provided nearest neighbor function, to search within the pre-built vectors :P 

In [53]:
ft.get_nearest_neighbors('こんにちは')

[(0.9167303442955017, 'こんばんは'),
 (0.9152794480323792, 'こんにちわ'),
 (0.8549860715866089, 'こんばんわ'),
 (0.7946215271949768, 'はじめまして'),
 (0.7212551236152649, 'おはよう'),
 (0.6740288734436035, 'どーも'),
 (0.6339334845542908, 'こんち'),
 (0.6219503283500671, 'どうも'),
 (0.6091426014900208, 'みなさん'),
 (0.5997785925865173, 'ゃにゃちは')]

Now we will convert all text features into word embeddings.
It might take you a few minutes.

In [54]:
features_vec = np.zeros((features.shape[0], ft.get_dimension()))

for i, sentence in enumerate(features):
    features_vec[i] = ft.get_word_vector(sentence)

In [55]:
print(features.shape)
# (sample, vector dim)
print(features_vec.shape)

(5167760,)
(5167760, 300)


Store the converted array into npz so we don't need to repeat this step every time

In [13]:
np.savez(npz_name, features=features, features_vec=features_vec)

-------
## Load the cache file

If you saved the npz file before, we can start from here directly.


In [65]:
with  np.load(npz_name, allow_pickle=True) as data:
    print("Keys in npz file: ", data.files)
    features = data["features"]
    features_vec = data["features_vec"]
print(features.shape)
print(features_vec.shape)

Keys in npz file:  ['features', 'features_vec']
(1000,)
(1000, 300)


Take a look a few sample

In [16]:
print(features[:5])
print(features_vec[:5][:10])

['中央ビルト工業労働組合 103-0014 東京都中央区日本橋蛎殻町1丁目32番4号泉山ビル302号'
 '有限会社雄樹 192-0352 東京都八王子市大塚659番地4' 'ルビノ株式会社 570-0016 大阪府守口市大日東町12番4-501号'
 'ニッポンレンタカーインターナショナル株式会社 101-0022 東京都千代田区神田練塀町3番地'
 'RITS合同会社 241-0814 神奈川県横浜市旭区中沢1丁目45番15号']
[[ 0.00111406  0.0046774   0.00639492 ... -0.00128364  0.00148696
   0.00602549]
 [ 0.00214144 -0.0007309   0.00914047 ... -0.00852792  0.00559978
   0.00463601]
 [-0.0039255   0.00277318  0.01046435 ... -0.00221889 -0.0067863
   0.0023125 ]
 [ 0.0019468  -0.00327     0.01153757 ... -0.01252608 -0.00367758
   0.0089357 ]
 [-0.00116498  0.00584311  0.00664053 ... -0.00388806  0.00300434
   0.00281819]]


-------------
## Build Annoy Index

Define the number of trees, which is the number of random projections used by "Annoy" to create the index. The number of trees is a hyperparameter that affects the accuracy of the ANN search. You may need to experiment with different values to find the optimal number for your dataset:

In [56]:
# Adjust it to optimize the search performance
n_trees = 100

Here we go! We will build the whole forest with the dataset. It will take you a while to build it up.

In [66]:
# Note: shape[1] is the dimension of the embedding
# You can also change the metric to other values, such as euclidean, angular...etc
index = annoy.AnnoyIndex(features_vec.shape[1], metric='angular')

for i, row in enumerate(features_vec):
    index.add_item(i, row)

index.build(n_trees)

True

Save the ann forest!

In [67]:
index.save('test_1000.ann')
# index.save('test_full.ann')

True

-----
## Test the inference

Now we have the ANN forest trained, let's take a look how it performs.


Load the trained ann.

In [68]:
index = annoy.AnnoyIndex(features_vec.shape[1], 'angular')
# index.load('test_full.ann')
index.load('test_1000.ann')

True

In [27]:
# pick some sample
test_samples = np.random.choice(features, 5, replace=False)
print(test_samples)

['有限会社アドバイザー 350-2201 埼玉県鶴ヶ島市富士見2丁目10番11号'
 '合同会社アクリエクト 870-0952 大分県大分市下郡北3丁目457番地の1フクシンビル2F'
 '有限会社ウエノ電設 675-1201 兵庫県加古川市八幡町宗佐467番地'
 '医療法人社団ふくろうの森 080-0019 北海道帯広市西九条南13丁目4番地1'
 '株式会社湧音工作 596-0031 大阪府岸和田市春木大小路町1番37号']


Prepare a function to infer with the classifier.

In [60]:
def search_ann(input_text: str): 
    test_emb = ft.get_word_vector(input_text)
    # Get 3 most closest index
    results, dists = index.get_nns_by_vector(test_emb, 3, search_k=-1, include_distances=True)
    # show the original label
    print("Input: ", input_text)
    for i, ind in enumerate(results):
        print(f"Distance : {dists[i]:0.4f} for entity: ", features[ind])
    print("=" * 100)

Now, we can do some test to see how it goes.

In [69]:
text_candidates = [
    # exactly same : '有限会社アドバイザー 350-2201 埼玉県鶴ヶ島市富士見2丁目10番11号'
    '有限会社アドバイザー 350-2201 埼玉県鶴ヶ島市富士見2丁目10番11号',
    # Remove part of company name :  '合同会社アクリエクト 870-0952 大分県大分市下郡北3丁目457番地の1フクシンビル2F'
    '合同会アクリ 870-0952 大分県大分市下郡北3丁目457番地の1フクシンビル2F',
    # Remove part of postal code and add wrong digit :  '有限会社ウエノ電設 675-1201 兵庫県加古川市八幡町宗佐467番地'
    '有限会社ウエノ電設 635-01 兵庫県加古川市八幡町宗佐467番地',
    # Remove part of address and add wrong text :  '医療法人社団ふくろうの森 080-0019 北海道帯広市西九条南13丁目4番地1'
    '医療法人社団ふくろうの森 080-0019 北海道帯西九条東丁目4番地',
    # Just break the whole text lol :  '株式会社湧音工作 596-0031 大阪府岸和田市春木大小路町1番37号'
    '株湧工作 596-000000 大阪府岸和田37号',
]

for i in text_candidates:
    search_ann(i)

Input:  有限会社アドバイザー 350-2201 埼玉県鶴ヶ島市富士見2丁目10番11号
Distance : 0.0000 for entity:  有限会社アドバイザー 350-2201 埼玉県鶴ヶ島市富士見2丁目10番11号
Distance : 0.9615 for entity:  みどりの風 141-0021 東京都品川区上大崎3丁目3-9-717
Distance : 0.9709 for entity:  有限会社フオトローグ 290-0157 千葉県市原市押沼690番地1ちはら台3-32-1-1ファミールハイツ9-401
Input:  合同会アクリ 870-0952 大分県大分市下郡北3丁目457番地の1フクシンビル2F
Distance : 0.3547 for entity:  合同会社アクリエクト 870-0952 大分県大分市下郡北3丁目457番地の1フクシンビル2F
Distance : 0.8528 for entity:  株式会社マイミチ 080-1408 北海道河東郡上士幌町字上士幌東1線234番地36プリマ上士幌C棟302号
Distance : 0.8907 for entity:  大分県農業協同組合中央会 870-0044 大分県大分市舞鶴町1丁目4番15号
Input:  有限会社ウエノ電設 635-01 兵庫県加古川市八幡町宗佐467番地
Distance : 0.7440 for entity:  有限会社ウエノ電設 675-1201 兵庫県加古川市八幡町宗佐467番地
Distance : 0.8426 for entity:  有限会社アイビーケセラ 668-0051 兵庫県豊岡市九日市上町467番地1
Distance : 0.8960 for entity:  有限会社マックス 658-0051 兵庫県神戸市東灘区住吉本町2丁目18番14号
Input:  医療法人社団ふくろうの森 080-0019 北海道帯西九条東丁目4番地
Distance : 0.7648 for entity:  医療法人社団ふくろうの森 080-0019 北海道帯広市西九条南13丁目4番地1
Distance : 0.9923 for entity:  浅野瀝青工業株式会社 065-0019 北海道札幌市東区北十九条東7丁目